In [1]:
import soccerdata as sd
from joblib import Parallel, delayed
import pandas as pd
from tqdm import tqdm

[03/12/24 23:31:38] INFO     No custom team name replacements found. You can configure these in       ]8;id=919157;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=710055;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\gfuen\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=77683;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=445695;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_config.py#283\283]8;;\
                             C:\Users\gfuen\soccerdata\config\league_dict.json.                                    

In [2]:
print(sd.ESPN.available_leagues())

['ARG-PrimeraDivision', 'BRA-Brasileirao', 'CHI-PrimeraDivision', 'ENG-Premier League', 'ESP-La Liga', 'FRA-Ligue 1', 'GER-Bundesliga', 'ITA-Serie A', 'NED-Eredivisie', 'POR-PrimeiraDivisao', 'RUS-PremierLeague', 'SCO-Premiership', 'TUR-SuperLig', 'USA-MLS']


In [ ]:
# Arreglar : ---
# En serie
league = "ESP-La Liga"
seasons = [i for i in range(2000, 2023)]
espn = sd.ESPN(leagues=league, seasons=seasons, no_cache=True)
schedule = espn.read_schedule()
schedule

In [ ]:
ids_con_problemas = [14654,14653,15868,16446,17917,17924,53989,56397]
leagues = sd.ESPN.available_leagues()

for l in leagues[1:2]:
    seasons = [i for i in range(2000, 2023)]
    espn = sd.ESPN(leagues=l, seasons=seasons, no_cache=True)
    schedule = espn.read_schedule()
    while True:
        try:
            match_ids = list(set(schedule.game_id.unique())- set(ids_con_problemas))
            df = espn.read_matchsheet(match_ids)
            df.to_csv(f"ESPN_{l}.csv")
            print(f"Liga {l} lista")
            break
        except ConnectionError as err:
            print(f"La liga {l} presenta otro error")
            id_broken = str(err).split("=")[-1][:-1]
            ids_con_problemas.append(int(id_broken))
        except:
            print(f"La liga {l} presenta otro error")



In [32]:
df = pd.DataFrame()
lista_de_años = [i for i in range(2000, 2023)][::-1]
ids_con_problemas = [14654, 14653, 15868, 16446, 17917, 17924, 53989, 56397]
leagues = ["ESP-La Liga", 'ARG-PrimeraDivision', 'BRA-Brasileirao', 'CHI-PrimeraDivision',
           'POR-PrimeiraDivisao', 'RUS-PremierLeague', 'SCO-Premiership', 'TUR-SuperLig', 'USA-MLS']
for l in leagues[0:5]:
    for a in lista_de_años:
        df = pd.DataFrame()
        try:
            espn = sd.ESPN(leagues=l, seasons=a, no_cache=True)
            schedule = espn.read_schedule()
            match_ids = list(set(schedule.game_id.unique()) -
                            set(ids_con_problemas))
            data = espn.read_matchsheet(match_ids)
            df = pd.concat([df, data.reset_index()])
            print(f"La liga {l}, año {a} listo")
        except err:
            print(f"La liga {l} presenta otro error, en el año {a}")
    df.to_csv(f"ESPN_{l}.csv")
    

[03/12/24 22:27:31] INFO     Saving cached data to C:\Users\gfuen\soccerdata\data\ESPN                ]8;id=630448;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=628295;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#91\91]8;;\

Año 2022 listo


[03/12/24 22:32:31] INFO     Saving cached data to C:\Users\gfuen\soccerdata\data\ESPN                ]8;id=263478;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=69292;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#91\91]8;;\

c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py:498: UserWarning: Season id "2021" is ambiguous: interpreting as "20-21"
  warnings.warn(msg, stacklevel=1)


Año 2021 listo


...

[03/12/24 23:30:06] ERROR    Error while scraping                                                    ]8;id=663681;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=435627;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#367\367]8;;\
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375. Retrying... (attempt 2 of 5).                                                
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\soccerdata\_common.py", line 347, in _download_and_save                            
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\requests\models.py", line 1021, in raise_for_status                                
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 502 Server Error: Bad Gateway for url:                 
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375                                                                               

                    ERROR    Error while scraping                                                    ]8;id=228661;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=248502;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#367\367]8;;\
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375. Retrying... (attempt 3 of 5).                                                
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\soccerdata\_common.py", line 347, in _download_and_save                            
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\requests\models.py", line 1021, in raise_for_status                                
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 502 Server Error: Bad Gateway for url:                 
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375                                                                               

                    ERROR    Error while scraping                                                    ]8;id=438996;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=73947;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#367\367]8;;\
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375. Retrying... (attempt 4 of 5).                                                
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\soccerdata\_common.py", line 347, in _download_and_save                            
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\requests\models.py", line 1021, in raise_for_status                                
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 502 Server Error: Bad Gateway for url:                 
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375                                                                               

                    ERROR    Error while scraping                                                    ]8;id=864930;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=817834;file://c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packages\soccerdata\_common.py#367\367]8;;\
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375. Retrying... (attempt 5 of 5).                                                
                             Traceback (most recent call last):                                                    
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\soccerdata\_common.py", line 347, in _download_and_save                            
                                 response.raise_for_status()                                                       
                               File                                                                                
                             "c:\Users\gfuen\AppData\Local\Programs\Python\Python310\lib\site-packag               
                             es\requests\models.py", line 1021, in raise_for_status                                
                                 raise HTTPError(http_error_msg, response=self)                                    
                             requests.exceptions.HTTPError: 502 Server Error: Bad Gateway for url:                 
                             http://site.api.espn.com/apis/site/v2/sports/soccer/esp.1/summary?event               
                             =402375                                                                               

NameError: name 'l' is not defined